In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as reader
import datetime as dt
import statsmodels.api as sm

In [2]:
end = dt.date(2019,12,31)
start = dt.date(2019,1,1)
stock = ['TSLA']

In [3]:
#Here's a look at our Tesla stock dataset
reader.get_data_yahoo(stock, start, end)

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,TSLA,TSLA,TSLA,TSLA,TSLA,TSLA
Date,,,,,,
2019-01-02,62.023998,62.023998,63.026001,59.759998,61.220001,58293000
2019-01-03,60.071999,60.071999,61.880001,59.476002,61.400002,34826000
2019-01-04,63.537998,63.537998,63.599998,60.546001,61.200001,36970500
2019-01-07,66.991997,66.991997,67.348000,63.549999,64.344002,37756000
2019-01-08,67.070000,67.070000,68.802002,65.403999,68.391998,35042500
...,...,...,...,...,...,...
2019-12-24,85.050003,85.050003,85.094002,82.538002,83.671997,40273500
2019-12-26,86.188004,86.188004,86.695999,85.269997,85.582001,53169500


In [4]:
#We are only interested in the 'Adj Close column'
stock_prices = reader.get_data_yahoo(stock, start, end)['Adj Close']
stock_prices.head()

Symbols,TSLA
Date,
2019-01-02,62.023998
2019-01-03,60.071999
2019-01-04,63.537998
2019-01-07,66.991997
2019-01-08,67.070000


In [5]:
#Calculating the stcok return
stock_prices['TSLAr'] = np.log(stock_prices['TSLA']/stock_prices['TSLA'].shift(1))
stock_prices.head()

Symbols,TSLA,TSLAr
Date,,
2019-01-02,62.023998,NaN
2019-01-03,60.071999,-0.031978
2019-01-04,63.537998,0.056094
2019-01-07,66.991997,0.052935
2019-01-08,67.070000,0.001164


In [6]:
#Let's drop the NaN values
stock_prices = stock_prices.dropna()
stock_prices.head()

Symbols,TSLA,TSLAr
Date,,
2019-01-03,60.071999,-0.031978
2019-01-04,63.537998,0.056094
2019-01-07,66.991997,0.052935
2019-01-08,67.070000,0.001164
2019-01-09,67.706001,0.009438


Getting the benchmark data and returns

In [7]:
#We are only interested in the 'Adj Close column'
bench = ['^GSPC']
benchmark = reader.get_data_yahoo(bench, start, end)['Adj Close']
benchmark.head()

Symbols,^GSPC
Date,
2019-01-02,2510.030029
2019-01-03,2447.889893
2019-01-04,2531.939941
2019-01-07,2549.689941
2019-01-08,2574.409912


In [8]:
#Calculating the stcok return
benchmark['GSPCr'] = np.log(benchmark['^GSPC']/benchmark['^GSPC'].shift(1))
benchmark = benchmark.dropna()
benchmark

Symbols,^GSPC,GSPCr
Date,,
2019-01-03,2447.889893,-0.025068
2019-01-04,2531.939941,0.033759
2019-01-07,2549.689941,0.006986
2019-01-08,2574.409912,0.009649
2019-01-09,2584.959961,0.004090
...,...,...
2019-12-24,3223.379883,-0.000195
2019-12-26,3239.909912,0.005115
2019-12-27,3240.020020,0.000034


Getting the Fama-French factors

In [9]:
factors = pd.read_csv('fama_factors_2019.csv')
factors = factors.rename(columns={'yyyymmdd':'Date'})
factors = factors[1:] #since our stock prices data begins on 1/3/2019
factors

,Date,Mkt-RF,SMB,HML,RF
1,1/3/2019,-2.45,0.40,1.21,0.010
2,1/4/2019,3.55,0.41,-0.70,0.010
3,1/7/2019,0.94,0.97,-0.77,0.010
4,1/8/2019,1.01,0.53,-0.64,0.010
5,1/9/2019,0.56,0.45,0.09,0.010
...,...,...,...,...,...
247,12/24/2019,0.01,0.37,-0.02,0.007
248,12/26/2019,0.48,-0.53,-0.02,0.007
249,12/27/2019,-0.10,-0.53,-0.07,0.007
250,12/30/2019,-0.57,0.17,0.59,0.007


In [10]:
#If we compare the two datasets, we can see that they have the same number of rows
print(factors.shape)
print(stock_prices.shape)
print(benchmark.shape)

(251, 5)
(251, 2)
(251, 2)


Now let's merge the two datasets. Our main problem is with the 'Date' columns which have different formats

In [11]:
#Fixing the Date index problem
factors['Date'] = pd.to_datetime(factors.Date)

In [12]:
#Fixed
factors

,Date,Mkt-RF,SMB,HML,RF
1,2019-01-03,-2.45,0.40,1.21,0.010
2,2019-01-04,3.55,0.41,-0.70,0.010
3,2019-01-07,0.94,0.97,-0.77,0.010
4,2019-01-08,1.01,0.53,-0.64,0.010
5,2019-01-09,0.56,0.45,0.09,0.010
...,...,...,...,...,...
247,2019-12-24,0.01,0.37,-0.02,0.007
248,2019-12-26,0.48,-0.53,-0.02,0.007
249,2019-12-27,-0.10,-0.53,-0.07,0.007
250,2019-12-30,-0.57,0.17,0.59,0.007


In [14]:
#Now merging
data = pd.merge(stock_prices, factors, on='Date')
data.head()

,Date,TSLA,TSLAr,Mkt-RF,SMB,HML,RF
0,2019-01-03,60.071999,-0.031978,-2.45,0.40,1.21,0.01
1,2019-01-04,63.537998,0.056094,3.55,0.41,-0.70,0.01
2,2019-01-07,66.991997,0.052935,0.94,0.97,-0.77,0.01
3,2019-01-08,67.070000,0.001164,1.01,0.53,-0.64,0.01
4,2019-01-09,67.706001,0.009438,0.56,0.45,0.09,0.01


In [15]:
#Adding the benchmark dataset
datas =pd.merge(data, benchmark, on='Date')
datas.head()

,Date,TSLA,TSLAr,Mkt-RF,SMB,HML,RF,^GSPC,GSPCr
0,2019-01-03,60.071999,-0.031978,-2.45,0.40,1.21,0.01,2447.889893,-0.025068
1,2019-01-04,63.537998,0.056094,3.55,0.41,-0.70,0.01,2531.939941,0.033759
2,2019-01-07,66.991997,0.052935,0.94,0.97,-0.77,0.01,2549.689941,0.006986
3,2019-01-08,67.070000,0.001164,1.01,0.53,-0.64,0.01,2574.409912,0.009649
4,2019-01-09,67.706001,0.009438,0.56,0.45,0.09,0.01,2584.959961,0.004090


In [18]:
#Rescaling The dataset to have the same scale
dataset = datas[['Date', 'TSLAr', 'Mkt-RF', 'SMB', 'HML', 'RF', 'GSPCr']] #The 'TSLA' is not useful anymore
dataset[['Mkt-RF', 'SMB', 'HML', 'RF']] = dataset[['Mkt-RF', 'SMB', 'HML', 'RF']]/100
dataset

C:\Users\USER\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Date,TSLAr,Mkt-RF,SMB,HML,RF,GSPCr
0,2019-01-03,-0.031978,-0.0245,0.0040,0.0121,0.00010,-0.025068
1,2019-01-04,0.056094,0.0355,0.0041,-0.0070,0.00010,0.033759
2,2019-01-07,0.052935,0.0094,0.0097,-0.0077,0.00010,0.006986
3,2019-01-08,0.001164,0.0101,0.0053,-0.0064,0.00010,0.009649
4,2019-01-09,0.009438,0.0056,0.0045,0.0009,0.00010,0.004090
...,...,...,...,...,...,...,...
246,2019-12-24,0.014281,0.0001,0.0037,-0.0002,0.00007,-0.000195
247,2019-12-26,0.013292,0.0048,-0.0053,-0.0002,0.00007,0.005115
248,2019-12-27,-0.001300,-0.0010,-0.0053,-0.0007,0.00007,0.000034
249,2019-12-30,-0.037113,-0.0057,0.0017,0.0059,0.00007,-0.005798


Rit−Rft = αit + β1*(RMt−Rft) + β2*SMBt + β3*HMLt + ϵit

In [20]:
dataset['TSLA-RF'] = dataset.TSLAr - dataset.RF
dataset['RMt-Rft'] = dataset.GSPCr - dataset.RF

C:\Users\USER\AppData\Local\Temp/ipykernel_6748/2740655017.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['TSLA-RF'] = dataset.TSLAr - dataset.RF


In [21]:
dataset

,Date,TSLAr,Mkt-RF,SMB,HML,RF,GSPCr,TSLA-RF,RMt-Rft
0,2019-01-03,-0.031978,-0.0245,0.0040,0.0121,0.00010,-0.025068,-0.032078,-0.025168
1,2019-01-04,0.056094,0.0355,0.0041,-0.0070,0.00010,0.033759,0.055994,0.033659
2,2019-01-07,0.052935,0.0094,0.0097,-0.0077,0.00010,0.006986,0.052835,0.006886
3,2019-01-08,0.001164,0.0101,0.0053,-0.0064,0.00010,0.009649,0.001064,0.009549
4,2019-01-09,0.009438,0.0056,0.0045,0.0009,0.00010,0.004090,0.009338,0.003990
...,...,...,...,...,...,...,...,...,...
246,2019-12-24,0.014281,0.0001,0.0037,-0.0002,0.00007,-0.000195,0.014211,-0.000265
247,2019-12-26,0.013292,0.0048,-0.0053,-0.0002,0.00007,0.005115,0.013222,0.005045
248,2019-12-27,-0.001300,-0.0010,-0.0053,-0.0007,0.00007,0.000034,-0.001370,-0.000036
249,2019-12-30,-0.037113,-0.0057,0.0017,0.0059,0.00007,-0.005798,-0.037183,-0.005868


In [24]:
#Let's run our Fama-French regression model

y = dataset['TSLA-RF'] #Our dependant variable

X = dataset[['RMt-Rft', 'SMB', 'HML']] #Dependant variables

X_sm = sm.add_constant(X)

In [25]:
model = sm.OLS(y, X_sm)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                TSLA-RF   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.130
Method:                 Least Squares   F-statistic:                     13.43
Date:                Mon, 28 Mar 2022   Prob (F-statistic):           3.81e-08
Time:                        20:55:12   Log-Likelihood:                 535.97
No. Observations:                 251   AIC:                            -1064.
Df Residuals:                     247   BIC:                            -1050.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0002      0.002      0.110      0.912      -0.003       0.004
RMt-Rft        1.1580      0.239      4.845      0.000       0.687       1.629
SMB            1.1360      0.403      2.818      0.005       0.342       1.930
HML           -0.1919      0.317     -0.606      0.545      -0.816       0.432
==============================================================================
Omnibus:                       59.175   Durbin-Watson:                   1.889
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              723.565
Skew:                          -0.471   Prob(JB):                    7.58e-158
Kurtosis:                      11.264   Cond. No.                         227.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""